In [3]:
using JuMP
using GLPK 
using Cbc

In [4]:
function bin_packing_mip(items, bin_capacity)
    model = Model(GLPK.Optimizer)
    n = length(items)
    max_bins = n  # En el peor de los casos, cada ítem puede ir en un contenedor separado
    
    # Variables
    @variable(model, x[1:max_bins, 1:n], Bin)
    @variable(model, y[1:max_bins], Bin)

    # Restricciones
    @constraint(model, [j=1:n], sum(x[i,j] for i in 1:max_bins) == 1)
    @constraint(model, [i=1:max_bins], sum(x[i,j] * items[j] for j in 1:n) <= bin_capacity * y[i])

    # Función objetivo
    @objective(model, Min, sum(y))

    optimize!(model)

    println("Status: ", termination_status(model))
    println("Objective value: ", objective_value(model))
    println("Número de contenedores utilizados: ", sum(value.(y)))
    
    return value.(y)
end

bin_packing_mip (generic function with 1 method)

In [15]:
items = [5, 10, 15, 20, 25, 30, 5, 2, 3, 4, 5, 20, 30]  
bin_capacity = 60 

# Ejecutar el algoritmo
num_bins_used = bin_packing_mip(items, bin_capacity)
println("Número de contenedores utilizados: $(sum(num_bins_used))")

Status: OPTIMAL
Objective value: 3.0
Número de contenedores utilizados: 3.0
Número de contenedores utilizados: 3.0


In [16]:
items = [3,2,3,2,2,2]
bin_capacity = 10 

num_bins_used = bin_packing_mip(items, bin_capacity)
println("Número de contenedores utilizados: $(sum(num_bins_used))")

Status: OPTIMAL
Objective value: 2.0
Número de contenedores utilizados: 2.0
Número de contenedores utilizados: 2.0


In [17]:
items = [2,5,4,7,1,3,8]
bin_capacity = 10 

num_bins_used = bin_packing_mip(items, bin_capacity)
println("Número de contenedores utilizados: $(sum(num_bins_used))")

Status: OPTIMAL
Objective value: 3.0
Número de contenedores utilizados: 3.0
Número de contenedores utilizados: 3.0


In [25]:
# Función que permite leer las instancias de bin packing desde un archivo
function read_bin_packing_instance(filename)
    problems = []
    open(filename, "r") do file
        lines = readlines(file)
        P = parse(Int, lines[1])  # Número de instancias de problemas
        index = 2
        for _ in 1:P
            problem_id = strip(lines[index])
            index += 1
            bin_capacity, num_items, best_known_solution = split(lines[index])
            bin_capacity = parse(Float64, bin_capacity)
            num_items = parse(Int, num_items)
            best_known_solution = parse(Int, best_known_solution)
            index += 1
            items = []
            for _ in 1:num_items
                push!(items, parse(Float64, strip(lines[index])))
                index += 1
            end
            push!(problems, (problem_id, bin_capacity, num_items, best_known_solution, items))
        end
    end
    return problems
end

read_bin_packing_instance (generic function with 1 method)

In [46]:
function bin_packing_cbc(bin_capacity, items)
    n = length(items)
    max_bins = n  # En el peor de los casos, cada ítem puede ir en un contenedor separado
    
    model = Model(Cbc.Optimizer)
    
    # Variables
    @variable(model, x[1:max_bins, 1:n], Bin)
    @variable(model, y[1:max_bins], Bin)

    # Restricciones
    @constraint(model, [j=1:n], sum(x[i,j] for i in 1:max_bins) == 1)
    @constraint(model, [i=1:max_bins], sum(x[i,j] * items[j] for j in 1:n) <= bin_capacity * y[i])

    # Función objetivo
    @objective(model, Min, sum(y))

    set_optimizer_attribute(model, "logLevel", 0)  
    
    elapsed_time = @elapsed begin
        optimize!(model)
    end
    
    if termination_status(model) == MOI.INFEASIBLE
        println("El problema no tiene solución factible.")
        return -1  # Otra forma de indicar que no se encontró solución
    end
    
    num_bins_used = sum(value.(y))
    
    return num_bins_used, elapsed_time

end

bin_packing_cbc (generic function with 1 method)

In [27]:
# Función para leer y describir todas las instancias de un archivo
function describe_problems(problems)
    for (problem_id, bin_capacity, num_items, best_known_solution, items) in problems
        println("Resolviendo problema: $problem_id, con $num_items de items, se espera $best_known_solution, $items items")
    end
    return problems
end

describe_problems (generic function with 1 method)

In [44]:
function solve_bin_packing_instances(problems)
    for (problem_id, bin_capacity, num_items, best_known_solution, items) in problems
        println("Resolviendo problema: $problem_id, con $bin_capacity de capacidad y con $num_items de items, se espera solución óptima: $best_known_solution, $items")
        num_bins_used, time_taken = bin_packing_cbc(bin_capacity, items)
        println("Número de contenedores utilizados: $num_bins_used")
        println("Tiempo tomado: $time_taken segundos")
        println()
    end
end

solve_bin_packing_instances (generic function with 1 method)

In [52]:
problems = read_bin_packing_instance("./instances/uniform.txt")
solve_bin_packing_instances(problems)


Resolviendo problema: uniform1, con 150.0 de capacidad y con 10 de items, se espera solución óptima: 2, Any[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.010098549 segundos

Resolviendo problema: uniform2, con 100.0 de capacidad y con 8 de items, se espera solución óptima: 2, Any[10.0, 20.0, 30.0, 10.0, 20.0, 30.0, 10.0, 20.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.004435026 segundos

Resolviendo problema: uniform3, con 120.0 de capacidad y con 6 de items, se espera solución óptima: 2, Any[15.0, 25.0, 20.0, 30.0, 25.0, 20.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.004081491 segundos

Resolviendo problema: uniform4, con 100.0 de capacidad y con 10 de items, se espera solución óptima: 4, Any[20.0, 20.0, 30.0, 40.0, 10.0, 30.0, 25.0, 25.0, 20.0, 20.0]
Número de contenedores utilizados: 3.0
Tiempo tomado: 0.006201876 segundos

Resolviendo problema: uniform5, con 80.0 de capacidad y con 12

In [48]:
problems = read_bin_packing_instance("./instances/triplets.txt")
solve_bin_packing_instances(problems)

Resolviendo problema: triplet1, con 100.0 de capacidad y con 9 de items, se espera solución óptima: 3, Any[25.0, 25.0, 25.0, 20.0, 20.0, 20.0, 15.0, 15.0, 10.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.006522468 segundos

Resolviendo problema: triplet2, con 80.0 de capacidad y con 6 de items, se espera solución óptima: 2, Any[25.0, 25.0, 15.0, 20.0, 20.0, 10.0]
Número de contenedores utilizados: 2.0
Tiempo tomado: 0.005947859 segundos

